In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import pandas as pd
import glob
import keras
import time
import PIL.Image
import pickle
import os
import random
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Input, MaxPooling2D, ZeroPadding2D, Conv2D, Flatten
from keras.models import Sequential, Model
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam, SGD
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras.layers import MaxPool2D, AveragePooling2D, GlobalAveragePooling2D
import numpy as np
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from zipfile import ZipFile
from datetime import timedelta
from io import BytesIO

In [6]:
train_labels = pd.read_csv('/content/drive/MyDrive/cassava-leaf-disease-classification/train.csv')

In [7]:
csv_df = pd.read_csv('/content/drive/MyDrive/cassava-leaf-disease-classification/train.csv')
train_dir = '/content/drive/MyDrive/cassava-leaf-disease-classification/train_tfrecords/*'
test_dir = '/content/drive/MyDrive/cassava-leaf-disease-classification/test_tfrecords/*'
train_list = glob.glob(train_dir)
test_list = glob.glob(test_dir)
print("train: " + str(len(train_list)) + "\ntest: " + str(len(test_list)))

train: 16
test: 1


In [8]:
labels = train_labels
labels.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [9]:
labels['label']=labels['label'].astype(str)

In [13]:
train_datagen = ImageDataGenerator(rescale=1./225,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.3,
    horizontal_flip=True,)

print(train_datagen)
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_dir = '/content/drive/MyDrive/cassava-leaf-disease-classification/train_images'

In [15]:
train_generator = train_datagen.flow_from_dataframe(dataframe=labels,
      directory=train_dir,
      subset='training',
      x_col="image_id",
      y_col="label",
      shuffle=True,
      target_size=(150,150),
      batch_size=32,
      class_mode='categorical')

valid_generator = train_datagen.flow_from_dataframe(dataframe=labels,
      directory=train_dir,
      subset='validation',
      x_col="image_id",
      y_col="label",
      shuffle=True,
      target_size=(150,150),
      batch_size=32,
      class_mode='categorical')

Found 14978 validated image filenames belonging to 5 classes.
Found 6419 validated image filenames belonging to 5 classes.


In [25]:
def make_model_1():
    inputs = Input(shape = (150,150,3))
    model = Conv2D(filters=32, kernel_size=(3, 3),activation='softmax', input_shape=(150,150,3))(inputs)
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Conv2D(filters=64, kernel_size=(3, 3), activation='softmax')(model)
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Conv2D(filters=128, kernel_size=(3, 3), activation='softmax')(model)
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Flatten()(model)
    model = Dense(512, activation = "relu")(model)
    out = Dense(5, activation = 'softmax')(model)
    model = Model(inputs=inputs, outputs=out)
    
    model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['acc'])
    return model

In [26]:
model = make_model_1()
print(model.summary())

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 34, 34, 128)       7385

In [30]:
history = model.fit(train_generator,steps_per_epoch=20,
                    validation_data=valid_generator,validation_steps=20,
                     epochs=20)

Epoch 1/20
20/20 [==============================] - 183s 9s/step - loss: 1.2076 - acc: 0.6094 - val_loss: 1.1396 - val_acc: 0.6313
Epoch 2/20
20/20 [==============================] - 201s 10s/step - loss: 1.2498 - acc: 0.6000 - val_loss: 1.2435 - val_acc: 0.5891
Epoch 3/20
20/20 [==============================] - 190s 10s/step - loss: 1.1647 - acc: 0.6234 - val_loss: 1.2079 - val_acc: 0.6172
Epoch 4/20
20/20 [==============================] - 179s 9s/step - loss: 1.1883 - acc: 0.6187 - val_loss: 1.2103 - val_acc: 0.6125
Epoch 5/20
20/20 [==============================] - 169s 9s/step - loss: 1.1763 - acc: 0.6266 - val_loss: 1.1672 - val_acc: 0.6266
Epoch 6/20
20/20 [==============================] - 162s 8s/step - loss: 1.1796 - acc: 0.6203 - val_loss: 1.2082 - val_acc: 0.6094
Epoch 7/20
20/20 [==============================] - 149s 8s/step - loss: 1.1951 - acc: 0.6156 - val_loss: 1.1501 - val_acc: 0.6281
Epoch 8/20
20/20 [==============================] - 145s 7s/step - loss: 1.1872 -